In [1]:
import sys

PWD = '../../..'
sys.path.append(PWD)

import numpy as np
import pandas as pd

import tensorflow as tf

tf.get_logger().setLevel('ERROR')
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
import keras_tuner as kt

from src.network_utils import count_params
from src.cross_validator import KerasTunerCrossValidator
from src.dataset import load_dataset_train_test, load_dataset_train_val
from src.models import unet_builder as bare_model_builder

In [2]:
PROJECT_NAME = 'unet'

PLANE = 2
CHANNEL = 11

OVERWRITE = False

DATASET_PATH = PWD + '/data/dataset/dataset.pkl'
TRIALS_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/tuner'
CROSSVAL_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/cross_val'

LR = 0.01
ES_MIN_DELTA = 0.01

N_EPOCHS = 3000
BATCH_SIZE = 2048
MAX_TRIALS = 30
EXECUTIONS_PER_TRIAL = 2

TOP_N = 5
CROSSVAL_N_CV = 5
CROSSVAL_N_EXEC = 2
LOSS_WEIGHT = 10_000

In [3]:
X_base_train, _, y_base_train_original, _ = load_dataset_train_test(PWD, PLANE, CHANNEL)
X_train, X_val, y_train_original, y_val_original = load_dataset_train_val(PWD, PLANE, CHANNEL)

X_base_train.shape, X_train.shape, X_val.shape

((22412, 24), (17929, 24), (4483, 24))

In [4]:
def gaussian_kernel(mu, sigma=1., n=24):
    x = np.arange(0, n)
    return np.exp(-(x - mu) ** 2 / (2 * sigma ** 2))


y_base_train = np.array([gaussian_kernel(y) for y in y_base_train_original])
y_train = np.array([gaussian_kernel(y) for y in y_train_original])
y_val = np.array([gaussian_kernel(y) for y in y_val_original])

y_base_train.shape, y_train.shape, y_val.shape

((22412, 24), (17929, 24), (4483, 24))

# Model

In [5]:
def model_builder(hp: kt.HyperParameters) -> keras.Model:
    hp_unet_depth = hp.Int("unet_depth", min_value=0, max_value=3, step=1, default=2)
    hp_n_conv_layers = hp.Int("n_conv_layers", min_value=1, max_value=3, step=1)
    hp_filters_mult = hp.Choice("conv_filters_mult", values=[1, 2, 4, 8, 16], default=4)
    hp_spatial_dropout = hp.Choice("conv_spatial_dropout", values=[0.0, 0.1, 0.2])
    hp_batch_normalization = hp.Boolean("batch_normalization", default=False)
    hp_input_batch_normalization = hp.Boolean("input_batch_normalization", default=False)

    model = bare_model_builder(hp_unet_depth, hp_n_conv_layers, hp_filters_mult, hp_spatial_dropout,
                               hp_batch_normalization, hp_input_batch_normalization)
    model.compile(loss='mse', optimizer=optimizers.Adam(LR), loss_weights=LOSS_WEIGHT)
    return model

In [6]:
model_builder(kt.HyperParameters()).summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 24)]         0           []                               
                                                                                                  
 reshape (Reshape)              (None, 24, 1)        0           ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 24, 32)       96          ['reshape[0][0]']                
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 12, 32)       0           ['conv1d[0][0]']                 
                                                                                              

In [7]:
model_callbacks = [
    callbacks.EarlyStopping(patience=60, min_delta=ES_MIN_DELTA),
    callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, patience=10)
]

# Bayesian tuner

In [8]:
bayesian_tuner = kt.BayesianOptimization(model_builder, objective='val_loss', executions_per_trial=EXECUTIONS_PER_TRIAL,
                                         max_trials=MAX_TRIALS, directory=TRIALS_DIR, project_name=PROJECT_NAME,
                                         overwrite=OVERWRITE)

bayesian_tuner.search(X_train, y_train, validation_data=[X_val, y_val], epochs=N_EPOCHS, callbacks=model_callbacks,
                      batch_size=BATCH_SIZE, verbose=3)

Trial 40 Complete [00h 00m 36s]
val_loss: 15.325746536254883

Best val_loss So Far: 3.1124417781829834
Total elapsed time: 02h 59m 32s


# Results

In [9]:
for i, hyperparameters in enumerate(bayesian_tuner.get_best_hyperparameters(TOP_N)):
    print(f'========== Model {i} ==========')
    print(hyperparameters.get_config()['values'])
    model_tmp = model_builder(hyperparameters)
    print('Number of parameters:', count_params(model_tmp))

========== Model 0 ==========
{'unet_depth': 3, 'n_conv_layers': 2, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.0, 'batch_normalization': False, 'input_batch_normalization': True}
Number of parameters: 367043
========== Model 1 ==========
{'unet_depth': 3, 'n_conv_layers': 2, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.0, 'batch_normalization': False, 'input_batch_normalization': True}
Number of parameters: 367043
========== Model 2 ==========
{'unet_depth': 2, 'n_conv_layers': 3, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.0, 'batch_normalization': False, 'input_batch_normalization': True}
Number of parameters: 113539
========== Model 3 ==========
{'unet_depth': 3, 'n_conv_layers': 3, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.0, 'batch_normalization': False, 'input_batch_normalization': True}
Number of parameters: 475011
========== Model 4 ==========
{'unet_depth': 3, 'n_conv_layers': 1, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.0, 'batch_normalizatio

# Cross-validation for top 5 models

In [10]:
cross_validator = KerasTunerCrossValidator(bayesian_tuner, X_base_train, y_base_train, model_builder,
                                           directory=CROSSVAL_DIR, project_name=PROJECT_NAME,
                                           n_epochs=N_EPOCHS, batch_size=BATCH_SIZE, n_top=TOP_N,
                                           es_min_delta=ES_MIN_DELTA, n_cv=CROSSVAL_N_CV, n_executions=CROSSVAL_N_EXEC,
                                           overwrite=OVERWRITE)
model_scores = cross_validator()

{'unet_depth': 3, 'n_conv_layers': 2, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.0, 'batch_normalization': False, 'input_batch_normalization': True}
Number of parameters: 367043
Got score: 3.1458 (3.1884, 3.1033)
Got score: 3.1899 (3.2336, 3.1461)
Got score: 2.9474 (2.9365, 2.9583)
Got score: 3.0511 (3.0705, 3.0316)
Got score: 3.1709 (3.1524, 3.1894)


{'unet_depth': 3, 'n_conv_layers': 2, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.0, 'batch_normalization': False, 'input_batch_normalization': True}
Number of parameters: 367043
Got score: 3.1372 (3.1747, 3.0997)
Got score: 3.0883 (3.1583, 3.0183)
Got score: 2.9470 (2.9471, 2.9470)
Got score: 3.0716 (3.0932, 3.0500)
Got score: 3.1391 (3.1579, 3.1204)


{'unet_depth': 2, 'n_conv_layers': 3, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.0, 'batch_normalization': False, 'input_batch_normalization': True}
Number of parameters: 113539
Got score: 3.1314 (3.1258, 3.1371)
Got score: 3.0596 (2.9995, 3.1198)
Got score: 2.9375 (2.9406, 2.9344)
Got score: 3.0459 (3.0569, 3.0349)
Got score: 3.2065 (3.1699, 3.2431)


{'unet_depth': 3, 'n_conv_layers': 3, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.0, 'batch_normalization': False, 'input_batch_normalization': True}
Number of parameters: 475011
Got score: 3.1521 (3.1205, 3.1837)
Got score: 5.7684 (8.2607, 3.2761)
Got score: 5.3568 (3.1757, 7.5379)
Got score: 3.4310 (3.0854, 3.7765)
Got score: 3.3565 (3.5115, 3.2015)


{'unet_depth': 3, 'n_conv_layers': 1, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.0, 'batch_normalization': False, 'input_batch_normalization': True}
Number of parameters: 259075
Got score: 3.1714 (3.1387, 3.2041)
Got score: 3.0343 (3.0164, 3.0521)
Got score: 2.9836 (2.9713, 2.9959)
Got score: 3.0753 (3.0785, 3.0721)
Got score: 3.1895 (3.1598, 3.2192)


In [11]:
mean_scores = [f"{np.mean(scores):0.2f}" for scores in model_scores.values()]
std_scores = [f"{np.std(scores):0.2f}" for scores in model_scores.values()]
n_params = [count_params(model_builder(hyperparameters)) for hyperparameters in
            bayesian_tuner.get_best_hyperparameters(TOP_N)]

df = pd.DataFrame({'mean': mean_scores, 'std': std_scores, 'n_params': n_params}, index=model_scores.keys())
df.index.name = 'Model'
df

,mean,std,n_params
Model,,,
0,3.10,0.09,367043
1,3.08,0.07,367043
2,3.08,0.09,113539
3,4.21,1.11,475011
4,3.09,0.08,259075
